In [212]:
import camelot as c
from camelot import utils
import os
import pandas as pd
import matplotlib.pyplot as plt
import pyscreenshot as ss
import pytesseract as tes
import cv2
import numpy as np
import regex as re

files = os.listdir(r"C:\Users\there\Documents\Code\Numismatics\OCR\pdfs\simple")
pre = r"C:\Users\there\Documents\Code\Numismatics\OCR\pdfs\simple\\"

tes.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\\tesseract.exe"

In [213]:
# for file in files:
#     pdf = pre + file
#     tables = c.read_pdf(pdf, "all")
#     for table in tables:
#         if table.parsing_report["accuracy"] > 0.90:
#             dir = f"./data/{file}"
#             if not os.path.exists(dir):
#                 os.mkdir(dir)
#             table.to_csv(dir + f"/{table.parsing_report['order']}.csv")

# Single-File

In [214]:
# pdf = "Farhi_Y_2018_Coins_from_the_2008_2009_Ex"
# testPDF = pre + pdf + ".pdf" # 100,600,500,100 ||| 476,200,788,631 ||| tlx tly brx bry ||| (0,0) in BL corner
# tables = c.read_pdf(testPDF, "7", flavor="lattice", line_scale=40, line_tol=8, table_areas=["100,600,500,100"]) # line tolerance issues, must be variable. if too big, wont run

### Remove Duplicate Headers

In [215]:
# overall_df = tables[0].df
# for table in tables[1:]:
#     cur_df = table.df

#     if cur_df[0][0] == overall_df[0][0]: # if non-initial table includes header again...
#         cur_df = cur_df.drop(0) # ...remove the header

#     overall_df = overall_df.merge(cur_df, how="outer")

In [216]:
# cur_group = ""
# new_col = []
# group_indices = []

# for index, row in overall_df.iterrows():
#     if sum(value != "" for value in row) == 1: # if only one cell filled --- indicates this is a grouping label/separator
#         cur_group = next(value for value in row if value != "")
#         group_indices.append(index)
#     else:
#         new_col.append(cur_group)

# for index in group_indices:
#     overall_df = overall_df.drop(index)

# overall_df = overall_df.assign(group=new_col) # append new groups column

In [217]:
# overall_df.to_csv("CUR.csv", index=False, header=False, encoding="utf-8") # need to add label for group depending on user input

In [218]:
# print(table.parsing_report)
# pdf = "Farhi_Y_2018_Coins_from_the_2008_2009_Ex"
# testPDF = pre + pdf + ".pdf" # 100,600,500,100 ||| 476,200,788,631 ||| LX HiY RX LowY ||| (0,0) in BL corner ||| swap 2-4
# tables = c.read_pdf(testPDF, "7", flavor="lattice", line_scale=40, line_tol=8, table_areas=["0,755,577,100"]) # line tolerance issues, must be variable. if too big, wont run
# table = tables[0]
# c.plot(table, kind="contour") # line joint, text grid contour, textedge (Lat, Both, Stream)

# Bounding box tool

In [219]:
import tkinter as tk
from tkinter import filedialog
import pymupdf as mu
from screeninfo import get_monitors
import ctypes

In [220]:
def get_page_number():
    image = np.array(ss.grab([800, 95, 880, 135]).convert('RGB')) # 800-850-880
    text = tes.image_to_string(image , config='digits')
    text = re.search(r'\d+', text)
    if text is None:
        return ""
    else: 
        return int(text.group())

In [221]:
def get_pdf_page_dimensions(pdf_path, page_number):
    
    document = mu.open(pdf_path)
    page = document.load_page(page_number-1)
    rect = page.rect
    width = rect.width
    height = rect.height

    return width, height

In [222]:
file = ""

def browser_to_pdf_coord(tlx, tly, brx, bry, dpi, page):

    scaleFactor = ctypes.windll.shcore.GetScaleFactorForDevice(0) / 100
    pdf_width, pdf_height = get_pdf_page_dimensions(file, page)
    factor = 72 / dpi 

    monitor = get_monitors()[0]
    width_res = monitor.width * factor
    height_res = monitor.height * factor    

    scrollBar = 18 * factor
    window_junk_above_pdf_reader = 155 * factor
    height_res -= window_junk_above_pdf_reader  
    scale = round(height_res / pdf_height, 2)  

    pdf_width *= scale
    width_res -= scrollBar # only if scrollBar present
    pdf_left = (width_res - pdf_width) / 2 # equivalently: (width_res / 2) - (pdf_width / 2)
    pdf_left /= scale
    pdf_bottom = 146 * factor / scale

    # converting screen dpi to PDF dpi (72)
    tlx *= factor
    tly *= factor
    brx *= factor
    bry *= factor

    tlx /= scale
    brx /= scale
    tly /= scale
    bry /= scale

    tlx -= pdf_left
    brx -= pdf_left
    tly -= pdf_bottom
    bry -= pdf_bottom

    tly = pdf_height - tly # flipping, since PDF coords have origin in bottom-left while cursor coords start in top-left
    bry = pdf_height - bry

    return [f"{tlx},{tly},{brx},{bry}"]

In [223]:
tlx, tly, brx, bry = -1, -1, -1, -1

def extract(tool): # upon pressing "confirm" after boxing the table
    dpi = round(tool.winfo_fpixels("1i")) # 96 for me, seems to be standard unless you're running this on a smart fridge
    page = tool.children.get("page_textbox").get("1.0", tk.END)
    page = re.search(r'\d+', page)

    reminder = tool.children.get("reminder")
    confirm_button = tool.children.get("confirm_button")
    page_textbox = tool.children.get("page_textbox")
    textbox_label = tool.children.get("textbox_label")
    capture_button = tool.children.get("capture_button")

    if page is None:
        tk.messagebox.showwarning(title="Oops!", message="Missing page number")
    else:
        reminder["text"] = "..."

        try:
            page = int(page.group())
            area = browser_to_pdf_coord(tlx, tly, brx, bry, dpi, page)
            
            table = c.read_pdf(f"{file}", str(page), flavor="lattice", line_scale=40, line_tol=8, table_areas=area)[0].df
            reminder["text"] = f"Success! Grabbed {len(table)} rows."
            
            
            capture_button["text"] = "Capture table"
            textbox_label.after(2000, textbox_label.destroy)
            page_textbox.after(2000, page_textbox.destroy)
            confirm_button.after(2000, confirm_button.destroy)
            reminder.after(2000, reminder.destroy)

            # c.plot(table, kind="contour") # line joint, text grid contour, textedge (Lat, Both, Stream)
            # plt.show()
        except Exception as e:
            # print(e)
            reminder["text"] = "Remember to fit the PDF to page!"
            reminder["font"] = ('Helvetica', 9, 'bold')
            tk.messagebox.showwarning(title="Oops!", message="Table not found. Ensure page number is correct, PDF is fit to page (vertically stretched), and that your box is fully within the PDF.")
    

def upload_file(capture_button, tool):
    file_path = filedialog.askopenfilename(filetypes=[("PDF files", "*.pdf"), ("All files", "*.*")])
    if file_path:
        # print(file_path)
        global file
        file = file_path
        tk.Grid.rowconfigure(tool, 1, weight=1)
        capture_button.grid(row=1, column=0, sticky="NSEW", columnspan=2)
        

In [224]:
def draw(event, canvas):
    global tlx, tly
    tlx = event.x
    tly = event.y
    canvas.create_oval(tlx-2, tly-2, tlx+2, tly+2, fill="Black")

def end_draw(event, transparent, canvas, tool):
    global brx, bry
    brx = event.x
    bry = event.y

    capture_button = tool.children.get("capture_button")
    
    confirm_button = tk.Button(tool, text="Confirm", name="confirm_button", command=lambda: extract(tool))
    
    page_textbox = tk.Text(tool, name="page_textbox", width=3, height=1)
    textbox_label = tk.Label(tool, text="Page #", name="textbox_label")

    canvas.create_rectangle(tlx, tly, brx, bry, outline="black", width=5)

    page = get_page_number()

    tk.Grid.rowconfigure(tool, 2, weight=1)
    textbox_label.grid(row=2, column=0, sticky="SNEW")
    page_textbox.grid(row=2, column=1, sticky="SNEW")

    if not re.search(r'\d+', page_textbox.get("1.0", tk.END)): # if empty
        page_textbox.insert("1.0", str(page))

    capture_button.config(text="Redraw?")
    transparent.after(1000, transparent.destroy)

    tk.Grid.rowconfigure(tool, 3, weight=1)
    confirm_button.grid(row=3, column=0, sticky="SNEW", columnspan=2)

    reminder = tk.Label(tool, text="Remember to fit the PDF to page!", name="reminder", font=('Helvetica', 9, 'bold'))
    reminder.grid(row=4, column=0)


In [225]:
def on_capture_button_click(tool):
    transparent = tk.Tk()
    transparent.overrideredirect(True)
    transparent.state('zoomed')
    canvas = tk.Canvas(transparent)
    canvas.pack(fill=tk.BOTH, expand=True)

    capture_button = tool.children.get("capture_button")
    capture_button.config(text="...")

    transparent.attributes("-alpha", 0.5)

    canvas.bind("<ButtonPress-1>", lambda event: draw(event, canvas))
    canvas.bind("<ButtonRelease-1>", lambda event: end_draw(event, transparent, canvas, tool))

In [ ]:
# def download_csv():


In [226]:
tool = tk.Tk()
tool.title("COINS Tool")

# download_button = tk.Button(tool, text=f"Download csv ({-1} items)", name="download_button" command=download_csv)

upload_button = tk.Button(tool, text="Upload PDF", name='upload_button', command=lambda: upload_file(capture_button, tool))

capture_button = tk.Button(tool, text="Capture table", name="capture_button", 
                           command=lambda: on_capture_button_click(tool))



tk.Grid.columnconfigure(tool, 0, weight=1)
tk.Grid.columnconfigure(tool, 1, weight=1)
tk.Grid.rowconfigure(tool, 0, weight=1)

upload_button.grid(row=0, column=0, sticky="NSEW", columnspan=2)

tool.mainloop()

# Time: 6:45-

End